### Transformando los datos

Rara vez los datos que necesitamos para trabajar vienen ya integrados en un único _dataset_ con toda la información que nos hace falta, o con la forma o estructura precisa para nuestros análisis. Más bien, lo normal será que tengamos conjuntos de datos cargados por separado de varias fuentes distintas, y que tengamos que combinarlos y adaptarlos o transformarlos de alguna manera antes de avanzar con cálculos y modelos.

Con Pandas todas estas tareas son realmente sencillas.

#### Concatenando datos

En uno de los ejemplos del apartado anterior ya usamos la función `pd.concat()`. Con esta función podemos combinar dos o más Series o DataFrames, concatenándolos por cualquiera de sus ejes, por filas o por columnas.

In [ ]:
# Preparamos dos DataFrame sencillos de ejemplo
df1 = DataFrame({'x' : np.arange(1,5), 'y' : np.arange(5, 9)})
df2 = DataFrame({'x' : np.arange(1,4), 'z' : np.arange(11, 14)})

df1

,x,y
0,1,5
1,2,6
2,3,7
3,4,8


In [ ]:
df2

,x,z
0,1,11
1,2,12
2,3,13


In [ ]:
# Concatenamos las filas de los dos DataFrames
# (equivalente a poner explicitamente la opción `axis='rows'`)
pd.concat([df1, df2])

,x,y,z
0,1,5.0,NaN
1,2,6.0,NaN
2,3,7.0,NaN
3,4,8.0,NaN
0,1,NaN,11.0
1,2,NaN,12.0
2,3,NaN,13.0


El argumento principal de `pd.concat()` es una lista con los DataFrames que queremos concatenar. Si no indicamos nada más, concatena las filas de todos los DataFrames. Esto es equivalente a poner de forma explícita la opción `axis = 'rows'`.

Fíjate que como el primer DataFrame no tiene columna `z`, en el resultado las filas que vienen de `df1` muestran el valor `NaN` para esa columna. De la misma forma, como el segundo DataFrame no tiene columna `y`, en el DataFrame final las filas que vienen de `df2` muestran el valor `NaN`para esa columna.

Probablemente también te has dado cuenta de que en el índice de filas (a la izquierda) hay valores repetidos, porque `pd.concat()` ha mantenido los índices de los DataFrames originales. Podemos pedirle que descarte estos índices originales y cree unos nuevos.

In [ ]:
pd.concat([df1, df2], ignore_index=True)

,x,y,z
0,1,5.0,NaN
1,2,6.0,NaN
2,3,7.0,NaN
3,4,8.0,NaN
4,1,NaN,11.0
5,2,NaN,12.0
6,3,NaN,13.0


Si lo que necesitamos es concatenar las columnas, basta con indicar la opción `axis = 'columns'`.

In [ ]:
pd.concat([df1, df2], axis = 'columns')

,x,y,x,z
0,1,5,1.0,11.0
1,2,6,2.0,12.0
2,3,7,3.0,13.0
3,4,8,NaN,NaN


Como ves, aparecen valores `NaN` en las dos últimas columnas de la fila 3, porque el DataFrame `df2` solo tenía valores de las filas 0 a la 2. Además, la columna `x` está duplicada, ya que la tenemos en los dos DataFrames. Es el mismo comportamiento que al concatenar filas.

Para lidiar con los nombres de filas o columnas duplicados, otra opción aparte de ignorar los índices originales es añadir un nivel por cada DataFrame que combinamos.

In [ ]:
# Añadir un nivel etiquetando los datos de cada DataFrame
pd.concat([df1, df2], axis = 'columns', keys = ["DF1","DF2"])

DF1     DF2      
    x  y    x     z
0   1  5  1.0  11.0
1   2  6  2.0  12.0
2   3  7  3.0  13.0
3   4  8  NaN   NaN

La operación de concatenar un objeto Series o DataFrame con otro es bastante común, así que Pandas incluye un método `append()` en ambas clases, y que nos puede servir de atajo.

In [ ]:
df1.append(df2)

,x,y,z
0,1,5.0,NaN
1,2,6.0,NaN
2,3,7.0,NaN
3,4,8.0,NaN
0,1,NaN,11.0
1,2,NaN,12.0
2,3,NaN,13.0


En el caso de DataFrames, `append()` concatena filas, no tiene opción para columnas. Ten en cuenta además que `append()` no modifica el DataFrame original, si no que devuelve uno nuevo copiando los datos.

#### Combinando datos: `merge` y `join`

A lo largo de la primera parte de este curso viste cómo en R se podían combinar objetos `data.frame` usando `merge` u otras funciones de librerías externas. Esta forma de combinar datos tabulares es análoga a las típicas operaciones `JOIN` del lenguaje SQL. Si has trabajado con bases de datos, el concepto te resultará muy familiar.

En Pandas disponemos del método `pd.merge()` como herramienta principal para combinar datos. Para mostrarte cómo funciona, primero carguemos un par de datasets.

In [ ]:
# Vamos a cargar un par de datasets, atento a los directorios

# Primero los datos de peliculas
peliculas = pd.read_csv("../U09_datasets/sample_movie_list.csv", sep=";")
 
peliculas.head()

,title,year,country,language,duration,director_name,budget,gross
0,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0
1,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0
2,District B13,2004.0,France,French,84.0,Pierre Morel,12000000.0,1197786.0
3,Metropolis,1927.0,Germany,German,145.0,Fritz Lang,6000000.0,26435.0
4,The Puffy Chair,2005.0,USA,English,85.0,Jay Duplass,15000.0,192467.0


In [ ]:
# y unos datos de valoración de los espectadores
valoracion = pd.read_csv("../U09_datasets/sample_movie_rating.csv", sep=";")
 
valoracion.head()

,title,imdb_score,fb_likes
0,Alice in Wonderland,6.5,24000
1,The Puffy Chair,6.6,297
2,The Damned United,7.6,0
3,Joyeux Noel,7.8,11000
4,Thor: The Dark World,7.1,63000


Ahora podemos cruzar los dos DataFrames con `pd.merge()`

In [ ]:
pd.merge(peliculas, valoracion).head()

,title,year,country,language,duration,director_name,budget,gross,imdb_score,fb_likes
0,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,6.5,24000
1,The Puffy Chair,2005.0,USA,English,85.0,Jay Duplass,15000.0,192467.0,6.6,297
2,The Damned United,2009.0,UK,English,98.0,Tom Hooper,10000000.0,449558.0,7.6,0
3,Joyeux Noel,2005.0,France,French,116.0,Christian Carion,22000000.0,1050445.0,7.8,11000
4,Thor: The Dark World,2013.0,USA,English,112.0,Alan Taylor,170000000.0,206360018.0,7.1,63000


El método `pd.merge()` identifica automáticamente las columnas que aparecen con el mismo nombre en los dos DataFrames y las usa como clave de unión. Si queremos indicar o restringir manualmente qué columnas usar como clave de unión, utilizamos la opción `on`.

In [ ]:
pd.merge(peliculas, valoracion, on=['title']).head()

,title,year,country,language,duration,director_name,budget,gross,imdb_score,fb_likes
0,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,6.5,24000
1,The Puffy Chair,2005.0,USA,English,85.0,Jay Duplass,15000.0,192467.0,6.6,297
2,The Damned United,2009.0,UK,English,98.0,Tom Hooper,10000000.0,449558.0,7.6,0
3,Joyeux Noel,2005.0,France,French,116.0,Christian Carion,22000000.0,1050445.0,7.8,11000
4,Thor: The Dark World,2013.0,USA,English,112.0,Alan Taylor,170000000.0,206360018.0,7.1,63000


Miremos un momento cuántas filas tenemos en cada DataFrame.

In [ ]:
print("Filas en peliculas:", peliculas.shape[0])
print("Filas en valoraciones:", valoracion.shape[0])
print("Filas en merge:", pd.merge(peliculas, valoracion).shape[0])

Filas en peliculas: 20
Filas en valoraciones: 12
Filas en merge: 12


`pd.merge()` por defecto devuelve solo las combinaciones de filas cuya clave aparece en ambos DataFrames (la intersección de claves, o `INNER JOIN` en términos `SQL`). En este caso, hay películas para las que no tenemos valoración, así que esas filas quedan descartadas.

Podemos controlar si queremos que se incluyan todas las filas del primer, segundo o ambos DataFrames a pesar de que las claves no tengan correspondencia en los dos. Lo hacemos con la opción `how`, que puede tomar estos valores

| Valor   | Descripción        | SQL |
|:--------|:-------------------|:----|
| `'inner'` | Incluye filas si la clave está en ambos DataFrames (opción por defecto) | `INNER JOIN` |
| `'left'`  | Incluye todas las filas del DataFrame a la izquierda; completa el derecho con `NaN` si faltan claves | `LEFT OUTER JOIN` |
| `'right'`  | Incluye todas las filas del DataFrame a la derecha; completa el izquierdo con `NaN` si faltan claves | `RIGHT OUTER JOIN` |
| `'outer'`  | Incluye todas las filas de ambos DataFrames; completa con `NaN` si faltan claves en uno u otro | `OUTER JOIN` |

Probemos con nuestros DataFrames.

In [ ]:
pd.merge(peliculas, valoracion, how='left').head()

,title,year,country,language,duration,director_name,budget,gross,imdb_score,fb_likes
0,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0,NaN,NaN
1,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,6.5,24000.0
2,District B13,2004.0,France,French,84.0,Pierre Morel,12000000.0,1197786.0,NaN,NaN
3,Metropolis,1927.0,Germany,German,145.0,Fritz Lang,6000000.0,26435.0,NaN,NaN
4,The Puffy Chair,2005.0,USA,English,85.0,Jay Duplass,15000.0,192467.0,6.6,297.0


Como ves, ahora aparecen películas en el cruce que antes quedaban descartadas por no tener valoración. En estas nuevas filas incluidas ahora, las columnas del segundo DataFrame toman valor `NaN`, es decir, ausente o nulo.

Algunas veces los nombres de las columnas a cruzar no aparecen igual en los DataFrames.

In [ ]:
generos = pd.read_csv("../U09_datasets/sample_movie_genres.csv", sep=";")
 
generos.head()

,movie_title,genre
0,Alice in Wonderland,Adventure
1,Alice in Wonderland,Family
2,Alice in Wonderland,Fantasy
3,Casino Royale,Action
4,Casino Royale,Adventure


En estos casos usamos los argumentos `left_on` y `right_on` para indicar explícitamente qué columnas usar como claves en cada DataFrame.

In [ ]:
pd.merge(peliculas, generos, left_on="title", right_on="movie_title").head()

,title,year,country,language,duration,director_name,budget,gross,movie_title,genre
0,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0,Snatch,Comedy
1,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0,Snatch,Crime
2,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Alice in Wonderland,Adventure
3,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Alice in Wonderland,Family
4,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Alice in Wonderland,Fantasy


El resultado contiene las columnas utilizadas como clave de ambos DataFrames, lo cual es redundante. Podemos descartar una de las columnas utilizando `drop()`.

In [ ]:
pd.merge(peliculas, generos, left_on="title", right_on="movie_title").drop("movie_title", axis="columns").head()

,title,year,country,language,duration,director_name,budget,gross,genre
0,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0,Comedy
1,Snatch,2000.0,UK,English,104.0,Guy Ritchie,6000000.0,30093107.0,Crime
2,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Adventure
3,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Family
4,Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Fantasy


También te habrás fijado en que los valores del primer DataFrame se repiten para cada valor del segundo. Tenemos una relación 1-N o _"1 a muchos"_ (una misma película puede estar clasificada con varios géneros).

Si los dos DataFrames tienen definido un índice común a nivel de filas, podemos utilizarlo para hacer el cruce.

In [ ]:
# Vamos a utilizar el título como índice en ambos DataFrame
peliculas.set_index("title", inplace=True)
generos.set_index("movie_title", inplace=True)

# y ahora cruzamos indicando que queremos usar 
# el `left_index` y el `right_index`
pd.merge(peliculas, generos, left_index=True, right_index=True).head()

,year,country,language,duration,director_name,budget,gross,genre
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Adventure
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Family
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Fantasy
Casino Royale,2006.0,UK,English,144.0,Martin Campbell,150000000.0,167007184.0,Action
Casino Royale,2006.0,UK,English,144.0,Martin Campbell,150000000.0,167007184.0,Adventure


Al igual que anteriormente con la concatenación, la operación de cruzar dos DataFrames por su índice es tan habitual que Pandas incluye un método más directo por comodidad: `join()`.

In [ ]:
# Podemos usar el método `join()` de un DataFrame
peliculas.join(generos).head()

,year,country,language,duration,director_name,budget,gross,genre
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Adventure
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Family
Alice in Wonderland,2010.0,USA,English,108.0,Tim Burton,200000000.0,334185206.0,Fantasy
Casino Royale,2006.0,UK,English,144.0,Martin Campbell,150000000.0,167007184.0,Action
Casino Royale,2006.0,UK,English,144.0,Martin Campbell,150000000.0,167007184.0,Adventure
